In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, CenterCrop, ToTensor, Normalize, Resize

import numpy as np

from PIL import Image
import cv2
import matplotlib.pyplot as plt

/home/ghdrnjs/anaconda3/envs/searle/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.5], std = [0.5])
])

transform_normal = Compose([
    ToTensor()
])

In [3]:
# torchvision에서 MNIST 데이터 Load

train_data = datasets.MNIST(
    root="./data/MNIST",
    train=True,
    download=True,
    transform=transform_normal
)

test_data = datasets.MNIST(
    root="./data/MNIST/",
    train=False,
    download=True,
    transform=transform_normal
)

In [5]:
train_dataset, val_dataset = random_split(train_data, [55000, 5000])

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [29]:
class ResNet50(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, out_channels=64, kernel_size=7, stride=2, padding=3) # 일반 RGB 이미지의 경우 in_channels = 3
        self.conv2_sub = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=1, padding=1)
        )
        self.conv2_max_pool = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_max_pool(x)
        print(f"MaxPool Layer 통과한 출력 shape : {x.shape}")
        y = self.conv2_sub(x)
        print(f"하나의 Sequential Layer를 통과한 출력 shape (Skip Connection 구현 전) : {y.shape}")
        x += y

        return x

In [30]:
resnet = ResNet50()
data_total = []
for batch in train_loader:
    training_data = batch[0]
    print(f"MNIST 학습 데이터 shape : {training_data.shape}")
    label = batch[1]
    forward = resnet(training_data)
    data_total.append(forward)
data_total = torch.vstack(data_total)    

MNIST 학습 데이터 shape : torch.Size([32, 1, 28, 28])
MaxPool Layer 통과한 출력 shape : torch.Size([32, 64, 6, 6])
하나의 Sequential Layer를 통과한 출력 shape (Skip Connection 구현 전) : torch.Size([32, 256, 6, 6])


RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1